<a href="https://colab.research.google.com/github/ZLL94/docs/blob/master/%E2%80%9Ccnn_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



##### TensorFlow 作者版权所有 2019。

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 卷积神经网络（CNN）

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/cnn">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    查看 TensorFlow 官网</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    在 Google Colab 中运行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/cnn.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    查看 GitHub 上的源代码</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/cnn.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />下载 notebook</a>
  </td>
</table>

本教程演示了如何训练简单的[卷积神经网络](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network)（CNN）来对 [CIFAR 图像](https://www.cs.toronto.edu/~kriz/cifar.html)进行分类。因为本教程使用 [Keras Sequential API](https://www.tensorflow.org/beta/guide/keras/overview)，所以我们构建和训练模型只需几行代码。

### 导入 TensorFlow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### 下载和处理 CIFAR10 数据集


数据集 CIFAR10 包含 10 类 60,000 个彩色图像，每个类中有 6,000 个。将数据集分为 50,000 个训练图像和 10,000 个测试图像。这些类是相互独立没有交集的。

In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# 将像素标准化，范围为 0 至 1 
train_images, test_images = train_images / 255.0, test_images / 255.0

### 验证数据

为了验证数据集是否正确，我们绘制训练集中的前 25 个图像，并在每个图像下方显示类名。


In [0]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # CIFAR 的标签正好是数组方便我们索引
    # The CIFAR labels happen to be arrays, 
    # which is why we need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### 构建卷积基础

下面的 6 行代码使用一个通用模式定义卷积基础：一些 [Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) 和 [MaxPooling2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) 层。CNN 输入数据的维度为（image_height，image_width，color_channels），忽略批次大小。如果您不熟悉这维度，color_channels 指的是（R，G，B）。在此示例中，我们把 CIFAR 图像的（32,32,3）维度作为 CNN 输入。因此我们将传递 input_shape 参数给第一层。

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


显示目前的模型结构。

In [0]:
model.summary()


上图，您可以看到每个 Conv2D 和 MaxPooling2D 层的输出都是 3 维的张量（高度，宽度，通道）。随着网络的愈加复杂，我们宽度和高度愈发变小。每个 Conv2D 层的输出通道数由第一个参数（例如：32 或 64）控制。通常，随着宽度和高度不断的减小，我们计算时可以在每个 Conv2D 层中增加更多地输出通道。

### 将密集层添加到顶层
To complete our model, we will feed the last output tensor from the convolutional base (of shape (3, 3, 64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current output is a 3D tensor. First, we will flatten (or unroll) the 3D output to 1D,  then add one or more Dense layers on top. CIFAR has 10 output classes, so we use a final Dense layer with 10 outputs and a softmax activation.

为了完成我们的模型，我们将最后的输出张量从卷积基（（3,3,64）的维度）馈送到一个或多个密集层中以执行分类。输入密集层的矢量要求是 1 维，而当前输出是 3 维张量。首先，我们将 3 维输出展开为 1 维，然后再在顶部添加一个或多个密集层。CIFAR 有 10 个输出类，因此我们在最后添加一个由 softmax 激活带 10 个输出的 Dense 层。

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))



这是我们模型的完整结构。

In [0]:
model.summary()


如你所见，在通过两个密集层之后，我们（3，3，64）维度的输出被展开为（576）维度的矢量。

### 编译、训练模型

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### 评价模型

In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images, test_labels)

In [0]:
print(test_acc)

### 后续学习计划


如您所见，我们简单的 CNN 已达到约 70％ 的测试精度。这几行代码还不错！

这里使用的结构没有任何优化。下一步，您可以设计不同的 CNN ，或者可以尝试使用 Keras Subclassing API 以及 GradidentTape 的凡是编写 CNN。你可以找到这样一个[例子](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/advanced.ipynb)。